In [2]:
import os, sys, shutil
from random import random
from glob import glob
from pydub import AudioSegment

In [5]:
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)


for path in glob('data/combinedv2/*/*'):
    split_path = path.split('\\')
    root, cls, filename = split_path[0], split_path[1], split_path[2]
    try:
        sound = AudioSegment.from_file(path, 'wav')
        # sound = sound.set_frame_rate(16000)
        normalized_sound = match_target_amplitude(sound, -20.0)
        normalized_sound_path = os.path.join(root + '_normalized', cls, filename)
        os.makedirs(os.path.dirname(normalized_sound_path), exist_ok=True)
        normalized_sound.export(''.join(normalized_sound_path.split('.')[:-1]) + '.wav', format="wav", parameters=["-sample_fmt", 's16', '-sample_rate', '16000'])
    except Exception as e:
        print(e)
        print(path)
        pass

Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 2021-11-18-git-85a6b7f7b7-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev2, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame

In [13]:
import hashlib

TILtrain = glob('data/TIL_train/*/*.wav')
TILtrain_hashes = {f: hashlib.sha256(open(f, 'rb').read()).hexdigest() for f in TILtrain}
other = glob('data/NCSE_sorted/*.wav')
other = {f: hashlib.sha256(open(f, 'rb').read()).hexdigest() for f in other}
for k, v in TILtrain_hashes.items():
    if v in CREMAD_hashes.values():
        print(k)

In [5]:
for path in glob('data/self_recorded_normalized/*/*'):
    # Convert all files to 16khz
    sound = AudioSegment.from_file(path, path.split('.')[-1])
    sound = sound.set_frame_rate(16000)
    sound_path = path.replace('self_recorded_normalized', 'self_recorded_normalized_16khz')
    os.makedirs(os.path.dirname(sound_path), exist_ok=True)
    sound.export(sound_path, format="wav", parameters=["-sample_fmt", 's16'])

In [1]:
# Handle torchaudio zero tensor bug

import torch
import torchaudio
import glob
import shutil
import os


for path in glob.glob('data/combinedv2_normalized/*/*.wav'):
    if not torch.all(torchaudio.load(path)[0][0][0]):
        print(path)
        shutil.move(path, path.replace('combinedv2_normalized', 'combinedv2_normalized_torch_bugged'))

data/combinedv2_normalized\angry\a01 (1).wav
data/combinedv2_normalized\angry\a01 (3).wav
data/combinedv2_normalized\angry\a02 (5).wav
data/combinedv2_normalized\angry\a03 (1).wav
data/combinedv2_normalized\angry\a09 (5).wav
data/combinedv2_normalized\angry\a11 (4).wav
data/combinedv2_normalized\angry\a13 (5).wav
data/combinedv2_normalized\angry\a14 (5).wav
data/combinedv2_normalized\angry\a17 (5).wav
data/combinedv2_normalized\angry\a20 (5).wav
data/combinedv2_normalized\angry\rab-f1-n1.wav
data/combinedv2_normalized\angry\rab-f2-b1.wav
data/combinedv2_normalized\angry\rab-f2-b2.wav
data/combinedv2_normalized\angry\rab-f2-b3.wav
data/combinedv2_normalized\angry\rab-f2-d1.wav
data/combinedv2_normalized\angry\rab-f2-d2.wav
data/combinedv2_normalized\angry\rab-f2-l1.wav
data/combinedv2_normalized\angry\rab-f2-l2.wav
data/combinedv2_normalized\angry\rab-f2-l3.wav
data/combinedv2_normalized\angry\rab-f2-l4.wav
data/combinedv2_normalized\angry\rab-f2-n1.wav
data/combinedv2_normalized\angry\

In [1]:
import os
import glob
import shutil
import pandas as pd

ans = pd.read_csv('answer.csv', usecols=[0, 1], names=['filename', 'label'])
# ans.columns = ['filename', 'label']
# ans.to_csv('answer.csv', index=False)
for path in glob.glob('data/TIL_test_normalized/*.wav'):
    label = ans[ans['filename'] == path.split(os.path.sep)[-1]]['label'].tolist()[0]
    new_path = os.path.join('data/TIL_ans', label, path.split(os.path.sep)[-1])
    os.makedirs(os.path.join('data/TIL_ans', label), exist_ok=True)
    shutil.copy(path, new_path)